# Background Information

### Dataset
The dataset contains the number of burglary reported in Sutton. It is downloaded from https://data.sutton.gov.uk/data-catalog-explorer/indicator/I2887/?view=table.

### Goal of current script
To practice data visualisation

### Content
1. Data cleaning
2. Asking questions
3. Data mining
4. Draw conclusions

# 1. Data Cleaning

In [62]:
# Import libraries
import pandas as pd
import math
from bokeh.plotting import figure, show # embed all elements of a graph
from bokeh.charts import output_notebook, BoxPlot

output_notebook()

Loading BokehJS ...

In [2]:
# Import data
df = pd.read_csv('Burglary_data.csv')

# Drop missing data
df = df.dropna()

# Check data type of each column
'''If the column contains 1 or more entries that are different from others, their type would be reported.'''
print(df.dtypes)

NAME               object
Jan-11 - Dec-11     int64
Jan-12 - Dec-12     int64
Jan-13 - Dec-13     int64
Jan-14 - Dec-14     int64
Jan-15 - Dec-15     int64
Jan-16 - Dec-16     int64
Jan-17 - Dec-17     int64
Jan-18 - Dec-18     int64
Jan-19 - Dec-19     int64
Jan-20 - Dec-20     int64
Jan-21 - Dec-21     int64
dtype: object


# Task 2 - Summary of data and possible trend

We will show the minimum, maximum, average and total number of burglary for each year. We also want to know if there is a trend over the years.

In [13]:
df.describe()

,Jan-11 - Dec-11,Jan-12 - Dec-12,Jan-13 - Dec-13,Jan-14 - Dec-14,Jan-15 - Dec-15,Jan-16 - Dec-16,Jan-17 - Dec-17,Jan-18 - Dec-18,Jan-19 - Dec-19,Jan-20 - Dec-20,Jan-21 - Dec-21
count,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000
mean,13.570248,12.975207,11.950413,11.545455,10.619835,10.314050,9.330579,9.801653,11.214876,8.404959,7.446281
std,8.130013,7.570186,7.166881,6.600505,5.990626,5.051457,5.337584,6.518716,6.859551,5.278223,4.593384
min,1.000000,0.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000
25%,8.000000,8.000000,7.000000,7.000000,6.000000,7.000000,6.000000,6.000000,7.000000,5.000000,4.000000
50%,12.000000,11.000000,10.000000,10.000000,9.000000,9.000000,8.000000,8.000000,10.000000,8.000000,7.000000
75%,17.000000,15.000000,15.000000,14.000000,13.000000,13.000000,12.000000,12.000000,15.000000,12.000000,9.000000
max,41.000000,43.000000,39.000000,42.000000,36.000000,27.000000,33.000000,55.000000,39.000000,39.000000,35.000000


In [254]:
'''Total number of burglary of each year'''
# Extract the name of each column and save them into a list
col = df.columns

# Compute the sum
total = []
for i in col:
    total.append(df[i].sum(axis=0))

# Add the result into the dataset
df.loc[len(df)] = total

# Change the name of the new row
df.at[121,'NAME'] = 'Total'

In [275]:
# Extract data of concern
source = df.iloc[121]
source = source.to_frame()
source['Name'] = source.index
source.columns = source.iloc[0]
source = source.drop(source.index[0])

In [303]:
# Plot line chart
p = figure(x_range=source.index.values, plot_height=450, plot_width=900)

p.title.text = 'Number of Burglary reported in Sutton (2011 - 2021)'
p.title.align = 'center'
p.title.text_font_size = '22px'

p.line(x='NAME', y='Total', source=source, width=2, color='blue')
p.circle(x='NAME', y='Total', source=source, size=14, color='gold')

p.xaxis.major_label_orientation = math.pi/8

show(p)

### Conclusion
The chart shows that number of burglary reported in Sutton has been decreasing over the decade, except in 2019 the number bounced back. The lockdown and economical impact of Covid-19 is probably the cause.

# Task 3 - Areas of concern

The dataset contains 120 Lower layer Super Output Area (LSOA). We will see which burglary rate are particularly of concern. We will answer this question with the flow below:

1. Add a column "Total" for each LSOA
2. Obtain the summary of the new column
3. Count LSOA lower/upper than the average
4. Identify outliers

In [67]:
# 1. Add a column
df3 = df.copy()
df3['Total'] = df3.sum(axis=1)
df3 = df3[['NAME', 'Total']]

# 2. Obtain the summary
df3.describe()

C:\Users\spong\AppData\Local\Temp/ipykernel_2548/1887882462.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df3['Total'] = df3.sum(axis=1)


,Total
count,121.000000
mean,117.173554
std,50.006613
min,48.000000
25%,83.000000
50%,105.000000
75%,142.000000
max,392.000000


In [78]:
'''Visualise the distribution of data'''
# Group the LSOA according to their number of burglary
gp1 = df3[(df3['Total'] <= 100)]['Total'].count()
gp2 = df3[(df3['Total'] <= 200) & (df3['Total'] > 100)]['Total'].count()
gp3 = df3[(df3['Total'] <= 300) & (df3['Total'] > 200)]['Total'].count()
gp4 = df3[(df3['Total'] <= 400) & (df3['Total'] > 300)]['Total'].count()

# Create a dataframe
dist = pd.DataFrame({'Group': ['0-100', '101-200', '201-300', '301-400'],
                     'Frequency': [gp1, gp2, gp3, gp4]})

,Group,Frequency
0,0-100,54
1,101-200,62
2,201-300,3
3,301-400,2


The figure shows that the distribution of the data is right/positively skewed. Therefore the outliers tend to be the LSOA that have high burglary number.

In [82]:
maxi, mini, avg, std = df3.max()[1], df3.min()[1], df3.mean(), df3.std()
up_out = avg + 3*std
low_out = avg - 3*std
maxi, mini, avg = int(maxi), int(mini), int(avg)

# Mean
count = df3[(df3['Total']>avg)].count()
print(f'Number of LSOA that has the number of burglary greater than the average: {count[1]}')
count = df3[(df3['Total']<=avg)].count()
print(f'Number of LSOA that has the number of burglary lower than or equal to the average: {count[1]}')

# Outliers
print(f'The upper and lower outliers are {round(up_out[0])} and {round(low_out[0])} correspondingly.')

Number of LSOA that has the number of burglary greater than the average: 44
Number of LSOA that has the number of burglary lower than or equal to the average: 77
The upper and lower outliers are 267 and -33 correspondingly.


C:\Users\spong\AppData\Local\Temp/ipykernel_2548/2234426925.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  maxi, mini, avg, std = df3.max()[1], df3.min()[1], df3.mean(), df3.std()


Since the lower outlier is negative which is smaller than our mean, it doesn't exist in our dataset. We will focus only on the outliers.

In [87]:
df3[(df3['Total'] > up_out[0])]

,NAME,Total
0,Sutton 011A,303
65,Sutton 012D,392


According to the figure above, there are 2 LSOA which number of burglary is particularly higher than others. They are Sutton 011A and 012D which have 303 and 392 cases of burglary reported throughout the decade correspondingly.